In [1]:
from bs4 import BeautifulSoup
import requests

def find_method_step(soup, recipe_detail):
    h3_text = "Method"
    h3 = soup.find("h3", string=h3_text)
    if h3:
        # 檢查 h3 的父層是否是具有特定 class 的元素
        parent_section = h3.find_parent("section", class_="recipe__method-steps")
        if parent_section:
            for element in parent_section:
                recipe_detail += element.text
        else:
            recipe_detail += "Can't found the recipe method."
    else:
        recipe_detail += "Can't found the recipe method."
    return recipe_detail

def find_ingredients(soup, recipe_detail):
    h2_text = "Ingredients"
    h2 = soup.find("h2", string=h2_text)
    if h2:
        # 檢查 h2 的父層是否是具有特定 class 的元素
        parent_section = h2.find_parent("section", class_="recipe__ingredients")
        if parent_section:
            for element in parent_section:
                recipe_detail += element.text
        else:
            recipe_detail += "Can't found the ingredient element."
    else:
        recipe_detail += "Can't found the ingredient element."
    return recipe_detail

def find_nutrition(soup, recipe_detail):
    # find nutrition information
    caption_text_partial = "Nutrition: per"
    caption = soup.find("caption", string=lambda text: text and caption_text_partial in text)
    if caption:
        table = caption.find_parent("table")
        # find nutrition table
        if table:
            # Do something with the table
            for element in table:
                recipe_detail += element.text
        else:
            recipe_detail += ("Can't found the nutrition information.")
    else:
        recipe_detail += ("Can't found the nutrition information.")
    return recipe_detail

def recipe_scraper(headers, link):
    recipe_detail = ""
    # find recipe webpage
    webpage = requests.get(f"https://www.bbcgoodfood.com{link}" ,headers=headers)
    soup = BeautifulSoup(webpage.content)

    # find ingredient element
    recipe_detail += find_ingredients(soup, recipe_detail)
    recipe_detail += "|"
    
    # find method step
    recipe_detail += find_method_step(soup, recipe_detail)
    recipe_detail += "|"

    # find nutrition
    recipe_detail += find_nutrition(soup, recipe_detail)
    
    return recipe_detail

def get_recipe(query):
    headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36"}
    recipe_list_page = requests.get(f'https://www.bbcgoodfood.com/search?q={query}' ,headers=headers)
    recipe_list_soup = BeautifulSoup(recipe_list_page.content)

    result = ""
    for article in recipe_list_soup.find_all("article")[:1]:
        recipe_info = "".join([article["data-item-name"], ":"])
        try:
            link = article.find("a")["href"]
            recipe_info += recipe_scraper(headers, link)
        except KeyError as e:
            recipe_info += recipe_info.join("Can't find recipe information.")
        result += recipe_info
    return result

In [2]:
query = "beef"
result = get_recipe(query)

In [3]:
tools_list = [
        {
            "type" : "function",
            "function" : {
                "name" : "get_recipe",
                "description" : "Fetch Food information based on a food or cusine name",
                "parameters" : {
                    "type" : "object",
                    "properties" : {
                        "query" : {"type": "string", "description": "Search query"},
                    },
                    "required":["query"]
                }
            }
        },
    ]

In [4]:
import os
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

instruction_words = "Role and Goal: Nutri Buddy is a Nutrition Assistant designed to assist users with information\
    based on their nutritional needs or questions. It specializes in providing details on food nutrition labels,\
    restaurant food nutrition, recommending specific restaurant foods, and offering recipe suggestions.\
    Guidelines: If you are asked about food-related recipes, use the 'food name' our 'cusine name' with the\
    'get_recipe' function."
# Create an Assistnat with a specific name
assistant = client.beta.assistants.create(
    name="Nutri Buddy",
    instructions=instruction_words,
    model="gpt-3.5-turbo-0125",
    tools=tools_list
)

# assistant = client.beta.assistants.retrieve("asst_Z4knqwOe9olvisDtzZGP0Fxo")

In [5]:
import time
# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you provide me a beef recipe with nutrition information?"
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="You are a nutrition assistant, answer the question as a nutritionist."
)

print(run.model_dump_json(indent=4))

while True:
    # Wait for 5 seconds
    time.sleep(5)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        break
    elif run_status.status == 'requires_action':
        print("requires action")
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            print(arguments)
            if func_name == "get_recipe":
                output = get_recipe(query=arguments['query'])
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": output
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)

{
    "id": "run_zbdZfdswIJ3pLU4CY7HG4drl",
    "assistant_id": "asst_DKREerjiduZKd1Lpxw3XXFNu",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1708364690,
    "expires_at": 1708365290,
    "failed_at": null,
    "file_ids": [],
    "instructions": "You are a nutrition assistant, answer the question as a nutritionist.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-3.5-turbo-0125",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_s8hEWeP1rE4Poz7YXgiSmG4s",
    "tools": [
        {
            "function": {
                "name": "get_recipe",
                "description": "Fetch Food information based on a food or cusine name",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Search quer